<a href="https://colab.research.google.com/github/n1c0e/Operations_Management-Gestion_de_operaciones-ICS3213/blob/main/Proyecto_IIC2433_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Entrega 3 </h2>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2022<br>    
    </p>
    <br>
</center>

<br>

---

## Setup


In [ ]:
from IPython.display import clear_output

!pip3 uninstall spacy
!pip3 install spacy

!spacy download en_core_web_lg
clear_output()
print('Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda')

Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda


In [ ]:
# Librerías
import pandas as pd
import numpy as np
from collections import defaultdict
import spacy

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier as DTC
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sb

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load("en_core_web_lg")

## Importar dataset

In [ ]:
data_train_url = "/content/train.dat"
data_test_url = "/content/test.dat"

DATA_TRAIN = pd.read_csv(data_train_url, sep='	', header=None, names=['label', 'abstract'])
DATA_TEST = pd.read_csv(data_test_url, sep='	', header=None, names=['abstract'])
DATA_TRAIN.head()

FileNotFoundError: ignored

In [ ]:
DATA_TEST.head()

## Preprocesamiento de datos

In [ ]:
# Funciones dadas la tarea 2
import string

all_stopwords = nlp.Defaults.stop_words

def remove_punctuation(text):
  text = [token for token in text if not token.is_punct]
  return text

def remove_stopwords(words):
  words = [word for word in words if not word in all_stopwords]
  return words

def lemmatize(words):
  words = [word.lemma_ for word in words]
  return words

def remove_non_alpha(words):
  words = [word for word in words if word.isalpha()]
  return words

def lower(words):
  words = [word.lower() for word in words]
  return words

def preprocess(text):

  doc = nlp(text)
  tokens = remove_punctuation(doc)
  tokens = remove_stopwords(tokens)
  tokens = lemmatize(tokens)
  tokens = remove_non_alpha(tokens)
  tokens = lower(tokens)

  return ' '.join(tokens).strip()


def sentence_vector(text):
  text = nlp(text)
  vectores = []
  for t in text:
    t_vector = t.vector
    vectores.append(t_vector)
  return np.array(vectores).sum(axis=0)/len(vectores)

Vectorizar train data

In [ ]:
# df_train = DATA_TRAIN.copy().drop(df_train.index[500:])
df_train = DATA_TRAIN.copy().sample(n=750)
df_train

In [ ]:
df_train['abstract'] = df_train['abstract'].apply(preprocess)
df_train.head()

In [ ]:
df_train['abstract'] = df_train['abstract'].apply(sentence_vector)

Vectorizar test data

In [ ]:
# df_test = DATA_TEST.copy().drop(df_test.index[50:])
# df_test['abstract'] = df_test['abstract'].apply(preprocess)
# df_test['abstract'] = df_test['abstract'].apply(sentence_vector)
# df_test.head()

### Dividir DATA en sets de train y de test aplicar PCA

Como solo data_train viene con labels, dividiremos este dataset en sets de train y de test con la función `train_test_split` de sklearn.model_selection.



In [ ]:
# Realiza aquí la separación balanceada en train y test
X = df_train['abstract']
y = df_train['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [ ]:
X_train.shape

In [ ]:
X_train = [i.tolist() for i in X_train.to_numpy()]
X_test = [i.tolist() for i in X_test.to_numpy()]

In [ ]:
y_train = y_train.to_numpy().tolist()

Ahora, se aplicará PCA para reducir el largo de los vectores retornados por NLP. Esto será una nueva data que se contrastará en la clasificaciones con la data original.

In [ ]:
pca_train = PCA(n_components=0.95, svd_solver='full')
X_train_pca = pca_train.fit_transform(X_train)
n_filas, n_columnas = X_train_pca.shape

pca_test = PCA(n_components=n_columnas, svd_solver='full')
X_test_pca = pca_test.fit_transform(X_test)
print(len(X_train_pca[0]), len(X_test_pca[0]))

## Clasificador

Linear SVC

In [ ]:
param_grid_linearSVC = {'C': [0.1, 1.0, 3.0, 100.0],\
              'loss': ['hinge', 'squared_hinge'],\
              'penalty': ['l1', 'l2']
              }

grid_linearSVC = GridSearchCV(LinearSVC(), param_grid_linearSVC, refit=True)
grid_linearSVC.fit(X_train, y_train)
clear_output()

In [ ]:
linearSVC = LinearSVC().set_params(**grid_linearSVC.best_params_)
linearSVC.fit(X_train, y_train)
y_predict_linearSVC = linearSVC.predict(X_test)

In [ ]:
print('hiperparámetros LinearSVC:', grid_linearSVC.best_params_)

In [ ]:
print("Matriz de confusión Linear SVC")
cm_linearSVC = confusion_matrix(y_test, y_predict_linearSVC, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_linearSVC).plot(cmap='magma')

In [ ]:
print(f"Accuracy LinearSVC = {metrics.accuracy_score(y_test, y_predict_linearSVC) * 100} %")

Linear SVC (PCA)

In [ ]:
param_grid_linearSVC = {'C': [0.1, 1.0, 3.0, 100.0],\
              'loss': ['hinge', 'squared_hinge'],\
              'penalty': ['l1', 'l2']
              }

grid_linearSVC = GridSearchCV(LinearSVC(), param_grid_linearSVC, refit=True)
grid_linearSVC.fit(X_train_pca, y_train)
clear_output()

In [ ]:
linearSVC = LinearSVC().set_params(**grid_linearSVC.best_params_)
linearSVC.fit(X_train_pca, y_train)
y_predict_linearSVC = linearSVC.predict(X_test_pca)

In [ ]:
print('hiperparámetros LinearSVC:', grid_linearSVC.best_params_)

In [ ]:
print("Matriz de confusión Linear SVC")
cm_linearSVC = confusion_matrix(y_test, y_predict_linearSVC, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_linearSVC).plot(cmap='magma')

In [ ]:
print(f"Accuracy LinearSVC = {metrics.accuracy_score(y_test, y_predict_linearSVC) * 100} %")

SVC


In [ ]:
param_grid_svc = {'C': [0.1,1, 10, 100],\
                  'gamma': [1,0.1,0.01,0.001],\
                  'kernel': ['rbf', 'poly', 'sigmoid']\
                  }

grid_svc = GridSearchCV(SVC(), param_grid_svc, refit=True)
grid_svc.fit(X_train, y_train)
clear_output()

In [ ]:
svc = SVC().set_params(**grid_svc.best_params_)
svc.fit(X_train, y_train)
y_predict_SVC = svc.predict(X_test)

In [ ]:
print('hiperparámetros SVC:', grid_svc.best_params_)

In [ ]:
print("Matriz de confusión SVC")
cm_SVC = confusion_matrix(y_test, y_predict_SVC, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_SVC).plot(cmap='magma')

In [ ]:
print(f"Accuracy SVC = {metrics.accuracy_score(y_test, y_predict_SVC) * 100} %")

SVC (PCA)

In [ ]:
param_grid_svc = {'C': [0.1,1, 10, 100],\
                  'gamma': [1,0.1,0.01,0.001],\
                  'kernel': ['rbf', 'poly', 'sigmoid']\
                  }

grid_svc = GridSearchCV(SVC(), param_grid_svc, refit=True)
grid_svc.fit(X_train_pca, y_train)
clear_output()

In [ ]:
svc = SVC().set_params(**grid_svc.best_params_)
svc.fit(X_train_pca, y_train)
y_predict_SVC = svc.predict(X_test_pca)

In [ ]:
print('hiperparámetros SVC:', grid_svc.best_params_)

In [ ]:
print("Matriz de confusión SVC")
cm_SVC = confusion_matrix(y_test, y_predict_SVC, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_SVC).plot(cmap='magma')

In [ ]:
print(f"Accuracy SVC = {metrics.accuracy_score(y_test, y_predict_SVC) * 100} %")

**PCA disminuye el accuracy. Mejor no utilizar.**

Random Forest

In [ ]:
param_grid_rfc = {'n_estimators': [10, 100, 500, 1000],\
                  'criterion': ['gini', 'entropy', 'log_loss']
                  }

grid_rfc = GridSearchCV(RFC(), param_grid_rfc, refit=True)
grid_rfc.fit(X_train, y_train)
clear_output()

In [ ]:
rfc = RFC().set_params(**grid_rfc.best_params_)
rfc.fit(X_train, y_train)
y_predict_rfc = rfc.predict(X_test)

In [ ]:
print('hiperparámetros RFC:', grid_rfc.best_params_)

In [ ]:
print("Matriz de confusión RFC")
cm_RFC = confusion_matrix(y_test, y_predict_rfc, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_RFC).plot(cmap='magma')

In [ ]:
print(f"Accuracy RFC = {metrics.accuracy_score(y_test, y_predict_rfc) * 100} %")

MLP

In [ ]:
param_grid_mlp = {
    'hidden_layer_sizes': [(50, 100, 200, 200, 100, 50)],
    'activation': ['logistic', 'relu'],
    'max_iter': [100, 300, 500, 800],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling']
}

grid_MLP = GridSearchCV(MLPClassifier(), param_grid_mlp, refit=True)
grid_MLP.fit(X_train, y_train)
clear_output()

In [ ]:
mlp = MLPClassifier().set_params(**grid_MLP.best_params_)
mlp.fit(X_train, y_train)
y_predict_MLP = mlp.predict(X_test)

In [ ]:
print('hiperparámetros MLP:', grid_MLP.best_params_)

In [ ]:
print("Matriz de confusión MLP")
cm_MLP = confusion_matrix(y_test, y_predict_MLP, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_MLP).plot(cmap='magma')

In [ ]:
print(f"Accuracy MLP = {metrics.accuracy_score(y_test, y_predict_MLP) * 100} %")

ADABoost

In [ ]:
param_grid_ada = {
    'estimator': [DTC(max_depth=2), DTC(max_depth=10), DTC(max_depth=100)],
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0, 10, 50, 100, 1000],
    'algorithm': ['SAMME', 'SAMME.R'],
}

grid_ada = GridSearchCV(AdaBoostClassifier(), param_grid_ada, refit=True)
grid_ada.fit(X_train, y_train)
clear_output()

In [ ]:
abc = AdaBoostClassifier().set_params(**grid_ada.best_params_)
abc.fit(X_train, y_train)
y_predict_abc = abc.predict(X_test)

In [ ]:
_print("Matriz de confusión ABC")
cm_abc = confusion_matrix(y_test, y_predict_abc, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_abc).plot(cmap='magma')

In [ ]:
print(f"Accuracy ABC = {metrics.accuracy_score(y_test, y_predict_abc) * 100} %")

XGBoost

In [ ]:
param_grid_xgb = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0, 10, 50, 100, 1000],
    'criterion': ['friedman_mse', 'squared_error']   
}

grid_xgb = GridSearchCV(GradientBoostingClassifier(), param_grid_xgb, refit=True)
grid_xgb.fit(X_train, y_train)
clear_output()

In [ ]:
xgbc = GradientBoostingClassifier().set_params(**grid_xgb.best_params_)
xgbc.fit(X_train, y_train)
y_predict_xgbc = xgbc.predict(X_test)
y_predict_xgbc2 = grid_xgb.predict(X_test)

In [ ]:
print("Matriz de confusión XGBC")
cm_xgbc = confusion_matrix(y_test, y_predict_xgbc, normalize="all")
cm_display = ConfusionMatrixDisplay(cm_xgbc).plot(cmap='magma')

In [ ]:
print(f"Accuracy XGBC = {metrics.accuracy_score(y_test, y_predict_xgbc) * 100} %")
print(f"Accuracy XGBC2 = {metrics.accuracy_score(y_test, y_predict_xgbc2) * 100} %")